In [20]:
import torch
from torch import nn
import torchvision
from torch.utils import data
from torchvision import transforms
import matplotlib.pyplot as plt
import random
import ipywidgets
import numpy as np
import math

In [21]:
def corr2d(X,K):
    h,w = K.shape
    a,b = X.shape
    Y = torch.zeros((a-h+1,b-w+1))
    for i in range(a-h+1):
        for j in range(b-w+1):
            Y[i,j] = (X[i:i+h,j:j+w] * K).sum()
    return Y

In [22]:
X = torch.arange(0.0,9.0).reshape((3,3))
X


tensor([[0., 1., 2.],
        [3., 4., 5.],
        [6., 7., 8.]])

In [23]:
K = torch.arange(0.0,4.0).reshape((2,2))
K

tensor([[0., 1.],
        [2., 3.]])

In [24]:
corr2d(X,K)

tensor([[19., 25.],
        [37., 43.]])

In [25]:
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))

    def forward(self,x):
        return corr2d(x, self.weight) + self.bias

In [26]:
X = torch.ones((6,8))
X[:,2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [27]:
K = torch.tensor([[1.0,-1.0]])

In [28]:
Y = corr2d(X,K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

In [29]:
# 纵向无法检测
corr2d(X,K.T)

tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

In [30]:
conv2d = nn.Conv2d(1,1,kernel_size=(1,2),bias=False)
X = X.reshape((1,1,6,8))
Y = Y.reshape((1,1,6,7))
X,Y

(tensor([[[[1., 1., 0., 0., 0., 0., 1., 1.],
           [1., 1., 0., 0., 0., 0., 1., 1.],
           [1., 1., 0., 0., 0., 0., 1., 1.],
           [1., 1., 0., 0., 0., 0., 1., 1.],
           [1., 1., 0., 0., 0., 0., 1., 1.],
           [1., 1., 0., 0., 0., 0., 1., 1.]]]]),
 tensor([[[[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
           [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
           [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
           [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
           [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
           [ 0.,  1.,  0.,  0.,  0., -1.,  0.]]]]))

In [31]:
trainer = torch.optim.SGD(conv2d.parameters(),lr=3e-2)

for i in range(10):
    Y_hat = conv2d(X)
    l = (Y_hat - Y) ** 2
    conv2d.zero_grad()
    l.sum().backward()
    trainer.step()
    print(f"epoch {i}, loss {l.sum()}")


epoch 0, loss 25.738128662109375
epoch 1, loss 10.62043571472168
epoch 2, loss 4.400112628936768
epoch 3, loss 1.8342745304107666
epoch 4, loss 0.7717913389205933
epoch 5, loss 0.3292282521724701
epoch 6, loss 0.1432374119758606
epoch 7, loss 0.06403674930334091
epoch 8, loss 0.029664158821105957
epoch 9, loss 0.014348656870424747


In [32]:
conv2d.weight.data.numpy()

array([[[[ 0.9912972 , -0.97515625]]]], dtype=float32)